In [2]:
!pip install xgboost

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 173.5 MB 159.9 MB/s            


In [34]:
import xgboost as xgb
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score, roc_curve, plot_roc_curve

In [4]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import matplotlib.image as mpimg
import os
from glob import glob
from IPython.display import Image
import random
%matplotlib inline

In [5]:
BASE_DIR = os.getcwd()
DATA_DIR = os.path.join(BASE_DIR, '..', 'data')
CheXpert_train_hidden_features_all = np.load(os.path.join(DATA_DIR,'CheXpert_train_hidden_features_all.npy'))
CheXpert_train_labels_all = np.load(os.path.join(DATA_DIR,'CheXpert_train_labels_all.npy'))
CheXpert_valid_hidden_features_all = np.load(os.path.join(DATA_DIR,'CheXpert_valid_hidden_features_all.npy'))
CheXpert_valid_labels_all = np.load(os.path.join(DATA_DIR,'CheXpert_valid_labels_all.npy'))
extra_valid_age_sex_df = pd.read_csv(os.path.join(DATA_DIR,'extra_valid_age_sex.csv'))
extra_valid_hidden_features = np.load(os.path.join(DATA_DIR,'extra_valid_hidden_features.npy'))
extra_valid_labels = np.load(os.path.join(DATA_DIR,'extra_valid_labels.npy'))
extra_valid_images = glob(os.path.join(DATA_DIR, 'extraValid', '*'))

In [6]:
label_headers = ['Cardiomegaly', 'Edema', 'Consolidation', 'Atelectasis', 'P. Effusion']
CheXpert_train_labels_all_df = pd.DataFrame(CheXpert_train_labels_all, columns=label_headers)
CheXpert_valid_labels_all_df = pd.DataFrame(CheXpert_valid_labels_all, columns=label_headers)
extra_valid_labels_df = pd.DataFrame(extra_valid_labels, columns=label_headers)

In [29]:
# Label smoothing. Single example can have multiple labels
total = 0
for i, row in enumerate(CheXpert_train_labels_all):
    count = 0
    for j, col in enumerate(row):
        if CheXpert_train_labels_all[i,j] == 1:
            count += 1
    if count >= 2:
        total += 1
print(total / i)

0.26183346537578384


In [53]:
gbc = GradientBoostingClassifier(n_estimators=200, learning_rate=0.1, max_depth=10, random_state=0)
multi_target_gbc = MultiOutputClassifier(gbc, n_jobs=-1)

In [ ]:
gbc_fitted = multi_target_gbc.fit(CheXpert_train_hidden_features_all, CheXpert_train_labels_all)

In [36]:
gbc_fitted.score(CheXpert_valid_hidden_features_all, CheXpert_valid_labels_all)

0.4188034188034188

In [38]:
y_pred = gbc_fitted.predict_proba(CheXpert_valid_hidden_features_all)

In [47]:
y_pred = np.transpose([pred[:, 1] for pred in y_pred])

In [49]:
auc = roc_auc_score(CheXpert_valid_labels_all, y_pred, average=None)

In [51]:
# lr 0.1, estimators 100, max_depth 1
print(np.mean(auc))

0.17854390912575177
